In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.cluster import KMeans
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path_to_augmented_dataset = "F:\\2022-2023\\SEM 1\\CV\\Dataset_1\\Augmented\\"
temp = ['Bowlegs', 'Knock-knee', 'Normal Knee']

In [ ]:
final = pd.DataFrame()
total_descriptors = 0
c = 0
for i in temp:
    path_to_folder = path_to_augmented_dataset+i
    for fname in os.listdir(path_to_folder):
        path_to_file = path_to_folder+"\\"+fname
        img = cv2.imread(path_to_file)
        cell_size = (16, 16)  # h x w in pixels
        block_size = (4, 4)  # h x w in cells
        nbins = 9  # number of orientation bins
        hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                          img.shape[0] // cell_size[0] * cell_size[0]),
                                _blockSize=(block_size[1] * cell_size[1],
                                            block_size[0] * cell_size[0]),
                                _blockStride=(cell_size[1], cell_size[0]),
                                _cellSize=(cell_size[1], cell_size[0]),
                                _nbins=nbins)
        descriptor = hog.compute(img)
        print(len(descriptor))
        total_descriptors += len(descriptor)
        out1 = pd.DataFrame(descriptor)
        csv_data = out1.to_csv(r'HOG\HOG_{}.csv'.format(i), mode='a', index=False)
print("total_descriptors = ", total_descriptors)

In [25]:
def read_data(path):
    data = pd.read_csv(path)
    return data.astype(np.float64)

In [26]:
p0 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_Bowlegs.csv'
p1 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_Knock-knee.csv'
p2 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_Normal Knee.csv'
pf = 'HOG\HOG_FINAL.csv'

data1 = data_f = read_data(p0)
print(data1.shape)
data2 = read_data(p1)
print(data2.shape)
data3 = read_data(p2)
print(data3.shape)

(3456959, 1)
(3615403, 1)
(3374136, 1)


In [ ]:
inertias = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(data1)
    inertias.append(kmeans.inertia_)

plt.plot(range(1,11), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [27]:
#performing kmeans on each class

kmeans1 = KMeans(n_clusters=6)
kmeans1.fit(data1)

kmeans2 = KMeans(n_clusters=6)
kmeans2.fit(data2)

kmeans3 = KMeans(n_clusters=6)
kmeans3.fit(data3)


KMeans(n_clusters=6)

In [ ]:
import pickle

with open("HOG_model1.pkl", "wb") as f:
    pickle.dump(kmeans1, f)
with open("HOG_model2.pkl", "wb") as f:
    pickle.dump(kmeans2, f)
with open("HOG_model3.pkl", "wb") as f:
    pickle.dump(kmeans3, f)

In [28]:
final = pd.DataFrame()
c = 0
for i in temp:
    data = []
    path_to_folder = path_to_augmented_dataset+i
    for fname in os.listdir(path_to_folder):
        path_to_file = path_to_folder+"\\"+fname
        img = cv2.imread(path_to_file)
        cell_size = (16, 16)  # h x w in pixels
        block_size = (4, 4)  # h x w in cells
        nbins = 9  # number of orientation bins
        hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                          img.shape[0] // cell_size[0] * cell_size[0]),
                                _blockSize=(block_size[1] * cell_size[1],
                                            block_size[0] * cell_size[0]),
                                _blockStride=(cell_size[1], cell_size[0]),
                                _cellSize=(cell_size[1], cell_size[0]),
                                _nbins=nbins)
        descriptor = hog.compute(img)
        out1 = pd.DataFrame(descriptor)
        array_double = np.array(out1, dtype=np.double)
        if i == temp[0]:
            a=kmeans1.predict(array_double)
        elif i == temp[1]:
            a = kmeans2.predict(array_double)
        else:
            a = kmeans3.predict(array_double)
        hist=np.histogram(a,bins=6)
        data.append(hist[0])
    Output = pd.DataFrame(data)
    Output["Class"] = c
    csv_data=Output.to_csv(r'HOG\HOG_FINAL_{}.csv'.format(i), mode='a', index=False)
    c += 1


C:\Users\neilk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\neilk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\neilk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\neilk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
C:\Users\neilk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warni

In [29]:
pf0 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_FINAL_Bowlegs.csv'
pf1 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_FINAL_Knock-knee.csv'
pf2 = r'F:\2022-2023\SEM 1\CV\CV_FINAL\HOG\HOG_FINAL_Normal Knee.csv'
pfd = 'HOG\HOG_FINAL.csv'
d1 = fd = read_data(pf0)
print(d1)
d2 = read_data(pf1)
print(d2)
d3 = read_data(pf2)
print(d3)
fd = fd.append(d2)
fd = fd.append(d3)
print(fd)
csv_data = fd.to_csv(pfd, mode='a', index=False)

          0      1      2      3      4      5  Class
0    1993.0  230.0  367.0  186.0  125.0  699.0    0.0
1    1990.0  224.0  323.0  215.0  106.0  742.0    0.0
2    1371.0  164.0  554.0  346.0  195.0  970.0    0.0
3    1826.0  211.0  383.0  254.0  155.0  771.0    0.0
4    2015.0  221.0  309.0  215.0  129.0  711.0    0.0
..      ...    ...    ...    ...    ...    ...    ...
955  1602.0  172.0  494.0  324.0  192.0  816.0    0.0
956  2696.0  230.0  162.0  134.0  107.0  271.0    0.0
957  1211.0  151.0  679.0  377.0  193.0  989.0    0.0
958  1865.0  218.0  410.0  251.0  138.0  718.0    0.0
959  2406.0  230.0  206.0  155.0  106.0  497.0    0.0

[960 rows x 7 columns]
          0       1      2      3      4       5  Class
0     387.0  1225.0  195.0  617.0  187.0   989.0    1.0
1     282.0  1724.0  224.0  419.0  195.0   756.0    1.0
2     406.0  1134.0  183.0  640.0  211.0  1026.0    1.0
3     375.0  1241.0  183.0  623.0  231.0   947.0    1.0
4     249.0  1827.0  218.0  412.0  169.0   725.0

C:\Users\neilk\AppData\Local\Temp\ipykernel_19180\954935783.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fd = fd.append(d2)
C:\Users\neilk\AppData\Local\Temp\ipykernel_19180\954935783.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fd = fd.append(d3)


In [30]:
data = read_data(pfd)
X = data.iloc[:, :-1]
print(X)
Y = data.iloc[:, -1]
print(Y)

# train test split
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, train_size=0.8)
print(train_X)
print(test_X)

           0      1      2      3       4      5
0     1993.0  230.0  367.0  186.0   125.0  699.0
1     1990.0  224.0  323.0  215.0   106.0  742.0
2     1371.0  164.0  554.0  346.0   195.0  970.0
3     1826.0  211.0  383.0  254.0   155.0  771.0
4     2015.0  221.0  309.0  215.0   129.0  711.0
...      ...    ...    ...    ...     ...    ...
2896   622.0  272.0   95.0  862.0  1212.0  537.0
2897  2184.0  147.0  240.0  296.0   528.0  205.0
2898  2229.0  159.0  235.0  289.0   472.0  216.0
2899  1262.0  203.0  187.0  639.0   995.0  314.0
2900  1079.0  191.0  184.0  637.0  1127.0  382.0

[2901 rows x 6 columns]
0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2896    2.0
2897    2.0
2898    2.0
2899    2.0
2900    2.0
Name: Class, Length: 2901, dtype: float64
           0       1      2      3      4      5
345   1460.0   199.0  507.0  295.0  152.0  987.0
340   1525.0   180.0  511.0  298.0  185.0  901.0
353   1496.0   166.0  552.0  314.0  196.0  876.0
2885  1490.0   22

In [ ]:
# fig, ax = plt.subplots(figsize=(12, 7))
# ax.spines['top'].set_visible(False)
# ax.spines['left'].set_visible(False)
# ax.spines['right'].set_visible(False)
# # adding major gridlines
# ax.grid(color='grey', linestyle='-', linewidth=0.25, alpha=0.5)
# ax.scatter(train_X, train_Y, color="#8C7298")
# plt.show()

# KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score

sc = StandardScaler()
train_X_KNN = sc.fit_transform(train_X)
test_X_KNN = sc.transform(test_X)

In [32]:
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

In [33]:
classifier.fit(train_X_KNN, train_Y)

KNeighborsClassifier()

In [35]:
y_pred = classifier.predict(test_X_KNN)

cm = confusion_matrix(test_Y, y_pred)
ac = accuracy_score(test_Y,y_pred)
print(cm)
print(ac)

[[174   0   0]
 [  0 209   0]
 [  0   0 198]]
1.0


# XgBoost

In [36]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
model = XGBClassifier()
sc = StandardScaler()
train_X = sc.fit_transform(train_X)
test_X = sc.transform(test_X)
model.fit(train_X, train_Y)
print(model)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)


In [37]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(test_X)
# evaluate predictions
accuracy = accuracy_score(test_Y, y_pred)
print(accuracy)

1.0


# SVM

In [38]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

classifier_SVM = SVC(kernel='linear')
# classifier_SVM = SVC(kernel='sigmoid')
# classifier_SVM = SVC(kernel='linear')

In [39]:
classifier_SVM.fit(train_X, train_Y)

SVC(kernel='linear')

In [40]:
predictions = classifier_SVM.predict(test_X)
cm = confusion_matrix(test_Y, predictions)
ac = accuracy_score(test_Y,y_pred)

In [24]:
print(cm, ac)

[[173   0   8]
 [  2 206   7]
 [  0   6 179]] 0.9948364888123924
